In [1]:
import json

news = None
with open('url2content.json') as f:
    buffer = f.read()
    news = json.loads(buffer)

index_url = []
for url in news:
    index_url.append(url)

In [33]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
 
# 加载词典 pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
 
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']
 
# 将 token 转为 vocabulary 索引
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# 定义句子 A、B 索引
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
 
# 将 inputs 转为 PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])


100%|██████████| 109540/109540 [00:01<00:00, 75945.21B/s]


AssertionError: 

In [30]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
 
# GPU & put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')
 
# 得到每一层的 hidden states 
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# 模型 bert-base-uncased 有12层，所以 hidden states 也有12层
print(encoded_layers[0].size())  #(layers, hiddens)
#assert len(encoded_layers) == 12


torch.Size([1, 14, 768])


In [37]:
import numpy as np
import jieba
import gensim
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from multiprocessing import Pool
from torch.utils.data.dataset import Dataset
class BiRNN(nn.Module):
    # add sequence_length
    def __init__(self, input_size, hidden_size, num_layers, num_classes, sequence_length):
        super(BiRNN, self).__init__()
        self.embedding = BertModel.from_pretrained('bert-base-chinese')
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    
    def forward(self, question_tokens, question_segments, content_tokens, context_segments):
        question,_ = model(question_tokens, question_segments)
        content,_ = model(content_tokens, context_segments)
        
        return question